<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [1]:
!nvidia-smi

Thu Feb 23 18:13:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-23 18:13:22--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-23 18:13:22 (117 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-23 18:13:26--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 142.250.1.128, 108.177.121.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-02-23 18:13:26 (116 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [6]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [7]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [9]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
test_data  = pd.read_csv("/content/test.csv")

In [11]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [12]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

http://t.co/AY6zzcUpnz Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/sC9dNS41Mc 1
Skinny Jeans are Hazardous for Your Health! #socialnews http://t.co/LTMa9xQXpx 1
Killing Black Babies at Planned Parenthood where's the demonstrations looting of PP burning down the buildings Black Babies Lives Matter 1
Expect gusty winds heavy downpours and lightning moving northeast toward VA now. http://t.co/jyxafD4knK 1
Severe hailstorm in progress over Northeast Logan County... #cowx http://t.co/XK9OwGV1O5 1


## Split Data into Training and Validation Sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [18]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [19]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [20]:
text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2,  230,    1, 2534,   11,  954,  822,    1, 1562, 1250,  313,
          11,  131, 1161,  101]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [24]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/M4TcZaawpT
tf.Tensor(
[[ 0.00225743  0.01408351  0.01257808 ... -0.03108845  0.00063939
  -0.01424851]
 [-0.01586976 -0.04086768  0.03611395 ...  0.00483803  0.04988423
   0.0194569 ]
 [ 0.00556316 -0.01828245  0.0013266  ... -0.03155348 -0.00347556
  -0.04159724]
 ...
 [-0.03065456 -0.0118536   0.00032669 ... -0.03331566 -0.01692767
  -0.0209437 ]
 [ 0.01683784  0.04521778  0.00760446 ...  0.01664282  0.02245196
   0.02626603]
 [-0.01231862  0.01608163  0.00079259 ... -0.00152559  0.0450821
  -0.04357896]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
base_line_score = model_0.score(val_sentences,val_labels)

In [28]:
base_line_score

0.7926509186351706

In [29]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [30]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [31]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [32]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [33]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [36]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230223-181332
Epoch 1/5
215/215 [==============================] - 27s 101ms/step - loss: 0.6357 - accuracy: 0.6437 - val_loss: 0.5764 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4752 - accuracy: 0.8203 - val_loss: 0.4821 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3477 - accuracy: 0.8727 - val_loss: 0.4576 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2628 - accuracy: 0.9076 - val_loss: 0.4569 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2020 - accuracy: 0.9314 - val_loss: 0.4671 - val_accuracy: 0.7900


In [37]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4671 - accuracy: 0.7900


[0.4670592248439789, 0.7900262475013733]

In [38]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
model_1_pred_probs[10]

array([0.1897772], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [41]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.7938260923446943,
 'recall': 0.7900262467191601,
 'f1': 0.7874906473794848}

## Visualizing Learned Embeddings

In [43]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [44]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [47]:
embed_weights.shape

(10000, 128)

In [48]:
embed_weights

array([[-0.02873396, -0.039572  , -0.01780594, ..., -0.07825426,
        -0.04693709, -0.12794673],
       [-0.03756204, -0.03169594, -0.02094365, ..., -0.05078234,
        -0.04630594, -0.05922903],
       [-0.04206709, -0.04682646, -0.02832044, ..., -0.04369123,
        -0.05132429, -0.0573655 ],
       ...,
       [-0.01844532, -0.04462112, -0.01840513, ..., -0.02908422,
        -0.01535224, -0.03571395],
       [-0.03628172, -0.04168401, -0.04049225, ...,  0.02339311,
        -0.01209052, -0.01299497],
       [-0.0225349 ,  0.00223549,  0.10045686, ..., -0.01622096,
        -0.04917612, -0.01572773]], dtype=float32)

## Recurrent Neural Network

### Model 2: LSTM

In [49]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x) # When we are stacking we need return_sequences = True
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [50]:
model_2.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [51]:
model_2_history = model_2.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_2_lstm")]
            )

Saving TensorBoard log files to: model_logs/model_2_lstm/20230223-181430
Epoch 1/5
215/215 [==============================] - 16s 47ms/step - loss: 0.3762 - accuracy: 0.8339 - val_loss: 0.4807 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2456 - accuracy: 0.9028 - val_loss: 0.5147 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1760 - accuracy: 0.9336 - val_loss: 0.5699 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1314 - accuracy: 0.9518 - val_loss: 0.7420 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0976 - accuracy: 0.9622 - val_loss: 1.0105 - val_accuracy: 0.7612


In [52]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[0.00343211],
       [0.92243487],
       [0.9997727 ],
       [0.01968332],
       [0.00248097],
       [0.9995043 ],
       [0.61594987],
       [0.99998987],
       [0.99998116],
       [0.63221955]], dtype=float32)

In [53]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [54]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
calculate_results(val_labels,model_2_preds)

{'accuracy': 76.11548556430446,
 'precision': 0.761395918264994,
 'recall': 0.7611548556430446,
 'f1': 0.7597317731418467}

### Model-3 GRU Powered RNN

In [56]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#If returns sequences and stackings are true
# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [57]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [58]:
model_3.compile(loss = "binary_crossentropy",metrics = ['accuracy'],optimizer = "adam")

In [59]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_3_gru")]
            )

Saving TensorBoard log files to: model_logs/model_3_gru/20230223-181516
Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.1987 - accuracy: 0.9188 - val_loss: 0.7390 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1037 - accuracy: 0.9626 - val_loss: 0.7581 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0822 - accuracy: 0.9717 - val_loss: 0.8149 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0704 - accuracy: 0.9746 - val_loss: 1.0127 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0641 - accuracy: 0.9752 - val_loss: 0.9080 - val_accuracy: 0.7743


In [60]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


array([[0.0050346 ],
       [0.78364617],
       [0.99956053],
       [0.05130297],
       [0.00172246],
       [0.9980038 ],
       [0.3543619 ],
       [0.9998574 ],
       [0.99982363],
       [0.34487915]], dtype=float32)

In [61]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [62]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [63]:
calculate_results(model_3_pred,val_labels)

{'accuracy': 77.42782152230971,
 'precision': 0.7959917678955798,
 'recall': 0.7742782152230971,
 'f1': 0.7782717544922271}

### Model-4 Bidirectional RNN

In [64]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation = "sigmoid",)(x)
model_4 = tf.keras.Model(inputs,outputs)

In [65]:
model_4.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                             

In [66]:
model_4.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [67]:
mode_4_history = model_4.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_4_bidirectional")],
)

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230223-181542
Epoch 1/5
215/215 [==============================] - 22s 51ms/step - loss: 0.1217 - accuracy: 0.9583 - val_loss: 0.8130 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0591 - accuracy: 0.9740 - val_loss: 1.2729 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0544 - accuracy: 0.9761 - val_loss: 0.9772 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0546 - accuracy: 0.9749 - val_loss: 1.0452 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0436 - accuracy: 0.9785 - val_loss: 1.4040 - val_accuracy: 0.7638


In [68]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(val_sentences)))

24/24 [==============================] - 3s 5ms/step


In [69]:
calculate_results(val_labels,model_4_preds)

{'accuracy': 76.37795275590551,
 'precision': 0.7680633930423381,
 'recall': 0.7637795275590551,
 'f1': 0.7602932743471209}

### Model-5: Conv1D 

In [70]:
embedding_text = embedding(text_vectorizer(['this is a test sentence for embedding']))

In [71]:
embedding_text

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03866538, -0.03268608, -0.00407241, ..., -0.07825134,
         -0.03179147, -0.0195193 ],
        [-0.02134806, -0.04658018, -0.03143543, ..., -0.04119225,
         -0.05422612, -0.04523468],
        [-0.03905785, -0.03528107, -0.01795032, ..., -0.03075664,
         -0.03767323, -0.03777858],
        ...,
        [-0.03438888, -0.04848133, -0.0133245 , ..., -0.04754689,
         -0.02898028, -0.10571648],
        [-0.03438888, -0.04848133, -0.0133245 , ..., -0.04754689,
         -0.02898028, -0.10571648],
        [-0.03438888, -0.04848133, -0.0133245 , ..., -0.04754689,
         -0.02898028, -0.10571648]]], dtype=float32)>

In [72]:
conv_1d = layers.Conv1D(filters = 32,
kernel_size = 5,
activation = "relu",
padding = "same"
)

In [73]:
conv_1d_output = conv_1d(embedding_text)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_text.shape, conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [74]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[1.05282497e-02, 5.68339638e-02, 3.26066557e-03, 1.15358353e-01,
        4.80286516e-02, 1.35842443e-01, 4.90820035e-02, 3.32707688e-02,
        1.16344854e-01, 1.67746693e-01, 3.04875076e-02, 6.71351552e-02,
        1.52116135e-01, 3.49009596e-02, 2.15626247e-02, 0.00000000e+00,
        4.82883342e-02, 7.23878443e-02, 6.00770973e-02, 9.28794742e-02,
        3.22773904e-02, 1.11963890e-01, 2.27052122e-02, 1.01909973e-04,
        2.13966575e-02, 4.90701646e-02, 5.74241467e-02, 6.56666607e-02,
        1.09258667e-03, 1.39953688e-01, 1.86744854e-02, 6.19299635e-02]],
      dtype=float32)>

In [75]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu",padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name = "model_5_conv1d")

In [76]:
model_5.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [77]:
mode_5_history = model_5.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_5_conv")],
)

Saving TensorBoard log files to: model_logs/model_5_conv/20230223-181638
Epoch 1/5
215/215 [==============================] - 10s 36ms/step - loss: 0.1424 - accuracy: 0.9593 - val_loss: 0.8763 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0753 - accuracy: 0.9720 - val_loss: 0.9768 - val_accuracy: 0.7520
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0619 - accuracy: 0.9753 - val_loss: 1.0617 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0548 - accuracy: 0.9794 - val_loss: 1.1043 - val_accuracy: 0.7507
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0496 - accuracy: 0.9799 - val_loss: 1.1581 - val_accuracy: 0.7441


In [78]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

24/24 [==============================] - 0s 3ms/step


In [79]:
calculate_results(val_labels,model_5_preds)

{'accuracy': 74.40944881889764,
 'precision': 0.7441190004660975,
 'recall': 0.7440944881889764,
 'f1': 0.7425086575551497}

### Model 6: TFHub Pretrained Sentence Encoder

In [80]:
import tensorflow_hub as hub

In [81]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [82]:
samples = embed([sample_sentence,"When you call universal sentence , turns into numbers"])

In [83]:
print(samples)

tf.Tensor(
[[-0.00165963  0.00170079  0.04788961 ...  0.00854664  0.00299551
  -0.04015192]
 [ 0.0568927  -0.08954596 -0.0283566  ... -0.00659815  0.05909768
  -0.01464346]], shape=(2, 512), dtype=float32)


In [84]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
            input_shape = [],
            dtype = tf.string,
            trainable = False,
            name = "USE"
        )

In [85]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(1,activation = "sigmoid"),
],name = "model_6_USE")

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [86]:
model_6.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [87]:
mode_6_history = model_6.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_6_use")],
)

Saving TensorBoard log files to: model_logs/model_6_use/20230223-181719
Epoch 1/5
215/215 [==============================] - 7s 17ms/step - loss: 0.6451 - accuracy: 0.7472 - val_loss: 0.6089 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.5782 - accuracy: 0.7924 - val_loss: 0.5602 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.5359 - accuracy: 0.7975 - val_loss: 0.5291 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5077 - accuracy: 0.8009 - val_loss: 0.5078 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4880 - accuracy: 0.8028 - val_loss: 0.4937 - val_accuracy: 0.7913


In [88]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_labels = tf.squeeze(tf.round(model_6_pred_probs))

24/24 [==============================] - 1s 9ms/step


In [89]:
calculate_results(val_labels,model_6_pred_labels)

{'accuracy': 79.13385826771653,
 'precision': 0.791408689554475,
 'recall': 0.7913385826771654,
 'f1': 0.7905092056482355}

### Model-7 : 10% of the Data

In [102]:
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels = train_labels[:train_10_percent_split]
print(train_sentences_10_percent)

In [104]:
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(100,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid"),
])
model_7.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [105]:
model_7.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_11 (Dense)            (None, 100)               51300     
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 256,849,225
Trainable params: 51,401
Non-trainable params: 256,797,824
_________________________________________________________________


In [109]:
mode_7_history = model_7.fit(
    train_sentences_10_percent,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_7_use")],
)

Saving TensorBoard log files to: model_logs/model_7_use/20230223-182227
Epoch 1/5
22/22 [==============================] - 4s 52ms/step - loss: 0.6606 - accuracy: 0.7007 - val_loss: 0.6339 - val_accuracy: 0.7533
Epoch 2/5
22/22 [==============================] - 1s 27ms/step - loss: 0.5708 - accuracy: 0.8088 - val_loss: 0.5653 - val_accuracy: 0.7756
Epoch 3/5
22/22 [==============================] - 1s 24ms/step - loss: 0.4860 - accuracy: 0.8321 - val_loss: 0.5140 - val_accuracy: 0.7822
Epoch 4/5
22/22 [==============================] - 0s 23ms/step - loss: 0.4275 - accuracy: 0.8394 - val_loss: 0.4926 - val_accuracy: 0.7822
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.3914 - accuracy: 0.8438 - val_loss: 0.4838 - val_accuracy: 0.7769


In [113]:
model_7_pred_probs = model_7.predict(val_sentences)

24/24 [==============================] - 1s 9ms/step


In [114]:
model_7_pred = tf.squeeze(tf.round(model_7_pred_probs))

In [115]:
calculate_results(val_labels,model_7_pred)

{'accuracy': 77.69028871391076,
 'precision': 0.7771036524564976,
 'recall': 0.7769028871391076,
 'f1': 0.7757784765658782}

### Uploading model in Tensorboard

In [120]:
!tensorboard dev upload --logdir ./model_logs \
--name "NLP Modelling" \
--description "Comparing different models"\
--one_shot

2023-02-23 18:29:02.077817: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 18:29:02.077941: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 18:29:02.077960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs

This TensorBoard wil

### Saving and Loading Model

In [121]:
model_6.save("model_6.h5")

In [123]:
loaded_model_6 = tf.keras.models.load_model(
  'model_6.h5',custom_objects = {"KerasLayer":hub.KerasLayer}
)

In [124]:
loaded_model_6.evaluate(val_sentences,val_labels)

24/24 [==============================] - 1s 20ms/step - loss: 0.4937 - accuracy: 0.7913


[0.49368390440940857, 0.7913385629653931]